In [1]:
from fastlane_bot import Config, ConfigDB, ConfigNetwork, ConfigProvider
from fastlane_bot.bot import CarbonBot
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer, T
from fastlane_bot.helpers import TxHelper
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonBot))
from fastlane_bot.testing import *
plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("2.0", __VERSION__)

Using default database url, if you want to use a different database, set the backend_url found at the bottom of manager_base.py
ConstantProductCurve v2.10.1 (07/May/2023)
CarbonBot v3-b2.1 (03/May/2023)
imported m, np, pd, plt, os, sys, decimal; defined iseq, raises, require


C:\Users\nicho\AppData\Local\Temp\ipykernel_41212\1801012949.py:8: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-dark')


Version = 3-b2.1 [requirements >= 2.0 is met]


# Testing the _run functions on TENDERLY [NBTest022]

## Tenderly Configuration

### Set up the bot and curves

In [2]:
C = Config.new(config=Config.CONFIG_TENDERLY)
assert C.DATABASE == C.DATABASE_POSTGRES
# assert C.POSTGRES_DB == "defaultdb"
assert C.NETWORK == C.NETWORK_TENDERLY
assert C.PROVIDER == C.PROVIDER_TENDERLY
bot = CarbonBot(ConfigObj=C)
assert str(type(bot.db)) == "<class 'fastlane_bot.db.manager.DatabaseManager'>"

In [8]:
C.w3.provider.endpoint_uri

'https://rpc.tenderly.co/fork/293439f0-a76e-4cbf-a6cf-2a366132500b'

In [3]:
CCm = bot.get_curves()
exch = {c.P("exchange") for c in CCm}
print("Number of curves:", len(CCm))
print("Number of tokens:", len(CCm.tokens()))
print("Exchanges:", exch)

Number of curves: 3543
Number of tokens: 2230
Exchanges: {'uniswap_v2', 'bancor_v2', 'bancor_v3', 'uniswap_v3', 'sushiswap_v2', 'carbon_v1'}


In [4]:
used_curves = [
    # '340282366920938463463374607431768211527-0', #actually deleted from carbon
    # '340282366920938463463374607431768211527-1', #actually deleted from carbon
    # '2722258935367507707706996859454145691730-1',
    # '1701411834604692317316873037158841057360-1',
    # '1701411834604692317316873037158841057360-0',
    # '9187623906865338513511114400657741709390-0',
    # '12590447576074723148144860474975423823947-0',
    # '13951577043758477001998358904702496669779-1'
]
subcurves = [x for x in CCm if x.cid not in used_curves]
no_0x0 = [x for x in subcurves if '0x0-1AD5' not in x.pair]
CCm2 = CPCContainer([x for x in no_0x0 if x.params.exchange not in ['bancor_v2']])
len(CCm2)
#print(CCm2)

3446

In [5]:
pairs = [x.pair.split('/') for x in CCm if x.params.exchange == 'carbon_v1']
len(set([x for itm in pairs for x in itm]))

27

In [6]:
pairs = [x.pair for x in CCm if x.params.exchange == 'carbon_v1']
len(set(pairs))

61

In [ ]:
# CCm.bycid("5784800237655953878877368326340059594782").p

In [ ]:
# CCm.bycid("5d5cebf33c0049258c40db6bc90b1558").p

In [ ]:
# CCm.bycid("9a52c5bdef1c402497d1c4f5d7501f61").p

In [ ]:
# CCm.bycid("12590447576074723148144860474975423823947-0").p

In [ ]:
# 1/CCm.bycid("12590447576074723148144860474975423823947-1").p

## Actual run

In [7]:
flt = [T.USDC, T.USDT, T.ETH, T.LINK, T.BNT, T.WBTC, T.DAI]
# bot._run(flashloan_tokens=flt, CCm=CCm2)

## Analytics

In [9]:
ops = bot._find_arbitrage_opportunities_carbon_multi_pairwise(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_ARBOPPS)

c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: Run

In [10]:
ops

(Decimal('35182.55345428887442789530260570693247224038913145705009810626506805419921875'),
                                             WETH-6Cc2     USDC-eB48
 969                                         27.744288 -51580.864657
 1701411834604692317316873037158841057376-1 -10.346786  14485.500000
 1701411834604692317316873037158841057378-1  -3.746250   4495.500008
 1701411834604692317316873037158841057375-1 -11.892857  17839.285736
 1701411834604692317316873037158841057377-1  -5.800645   6960.774185
 1701411834604692317316873037158841057360-1  -5.032132   7799.804718
 AMMIn                                       27.744288  51580.864647
 AMMOut                                     -36.818670 -51580.864657
 TOTAL NET                                   -9.074382     -0.000010,
 [{'cid': '969',
   'tknin': 'WETH-6Cc2',
   'amtin': 27.744288000278175,
   'tknout': 'USDC-eB48',
   'amtout': -51580.86465716362,
   'error': None},
  {'cid': '1701411834604692317316873037158841057376-1',
   'tknin'

In [ ]:
# ord = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_ORDSCAL)
# ord

In [ ]:
# ti = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_TI)
# ti

In [ ]:
# ti

In [ ]:
# ops = bot._find_arbitrage_opportunities(flashloan_tokens=flt, CCm=CCm, result=bot.AO_CANDIDATES)
# ops

In [ ]:
# candf = pd.DataFrame(ops).sort_values(by=0, ascending=False)
# candf.reset_index(inplace=True, drop=True)
# candf.iloc[5][1]

In [ ]:
# candf.iloc[0][0]

In [ ]:
aggregated = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_AGGTI)
#aggregated

for trade in aggregated:
    print(trade.amtin, trade.tknin, trade.amtout, trade.tknout)


In [ ]:
exact = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_EXACT)
# exact

In [ ]:
for trade in exact:
    print(trade.amtin, trade.tknin, trade.amtout, trade.tknout)

In [ ]:
# assert False

In [ ]:
# agg = bot._run(flashloan_tokens=flt, CCm=CCm, result=bot.XS_AGGTI)
# agg

In [ ]:
# agg[0].raw_txs

In [ ]:
# agg[1].raw_txs

In [ ]:
exact[0].amtin_wei, exact[0].amtout_wei

In [ ]:
exact[1].amtin_wei, exact[1].amtout_wei

In [ ]:
route_struct, flashloan_amount, flashloan_token_address = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_ROUTE)

In [ ]:
route_struct, flashloan_amount, flashloan_token_address

# Tenderly Tx Submission

In [ ]:
# C.w3.provider.endpoint_uri

In [ ]:
# C.w3.eth.getBlock(C.w3.eth.block_number).timestamp

In [ ]:
# C.provider.CARBON_CONTROLLER_CONTRACT.strategy(13951577043758477001998358904702496669788)

In [ ]:
### This to manually submit
bot._validate_and_submit_transaction_tenderly(
        ConfigObj = C,
        route_struct = route_struct,
        src_amount = flashloan_amount,
        src_address = flashloan_token_address,
            )

# MAINNNET Preferred Tx Submission Method

In [ ]:
# from fastlane_bot.helpers import TxHelpers

In [ ]:
# pool = (
#     bot.db.get_pool(exchange_name=bot.ConfigObj.BANCOR_V3_NAME, pair_name='BNT-FF1C/ETH-EEeE')
# )
# bnt_eth = (int(pool.tkn0_balance), int(pool.tkn1_balance))

# # Init TxHelpers
# tx_helpers = TxHelpers(ConfigObj=bot.ConfigObj)
# # Submit tx



In [ ]:
# tx_helpers.validate_and_submit_transaction(route_struct=route_struct, src_amt=flashloan_amount,
#                                                     src_address=flashloan_token_address, bnt_eth=bnt_eth,
#                                                     expected_profit=Decimal('1000'), safety_override=True)

# MAINNET Forced Tx Submission Method

In [ ]:
# # Initialize tx helper
# tx_helper = TxHelper(
#     usd_gas_limit=55,
#     ConfigObj=bot.ConfigObj,
#     gas_price_multiplier = 1
# )
# # # Submit the transaction
# # tx_helper.submit_flashloan_arb_tx(
# #     arb_data=route_struct,
# #     flashloan_token_address=flashloan_token_address,
# #     flashloan_amount=flashloan_amount,
# #     verbose=True,
# # )

In [ ]:
# tx_helper.gas_limit